In [1]:
import pandas as pd
import openpyxl
import os
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pyparsing import col
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm 
import re

In [9]:
#검색할 식당 데이터

df = pd.DataFrame(columns=['name', 'naverURL'])

items = ['만년곱창', '홍리마라탕', '히글리', '레모노이']

df['name'] = items
df

,name,naverURL
0,만년곱창,NaN
1,홍리마라탕 권선점,NaN
2,히글리,NaN
3,레모노이,NaN


In [10]:
# 식당 url 얻기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

res = driver.page_source  # 페이지 소스 가져오기
soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다

# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    res
    soup

for i, keyword in enumerate(df['name'].tolist()):
    # 검색 url 만들기
    naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place'  
    # 검색 url 접속 = 검색하기
    driver.get(naver_map_search_url)
    time.sleep(2) 
    # 검색 프레임 변경
    driver.switch_to.frame("searchIframe")
    time.sleep(1) 
     
    
    try:     
        #식당 정보가 있다면 첫번째 식당의 url을 가져오기
        
        if len(driver.find_elements(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li')) != 0:   
            #식당 정보 클릭        
            driver.execute_script('return document.querySelector("#_pcmap_list_scroll_container > ul > li:nth-child(1) > div.ouxiq > a:nth-child(1) > div").click()')
            time.sleep(2)
            
            # 검색한 플레이스의 개별 페이지 저장
            tmp = driver.current_url  
            res_code = re.findall(r"place/(\d+)", tmp)
            final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#' 
        
            print(final_url)
            df['naverURL'][i]=final_url 
        
    except: 
        df['naverURL'][i]= ''
        print('none') 
    
driver.close()
    #df.to_csv('url_completed.csv', encoding = 'utf-8-sig')

df

https://pcmap.place.naver.com/restaurant/1476611616/review/visitor#
none
https://pcmap.place.naver.com/restaurant/1887134163/review/visitor#
https://pcmap.place.naver.com/restaurant/1972327571/review/visitor#


,name,naverURL
0,만년곱창,https://pcmap.place.naver.com/restaurant/14766...
1,홍리마라탕 권선점,
2,히글리,https://pcmap.place.naver.com/restaurant/18871...
3,레모노이,https://pcmap.place.naver.com/restaurant/19723...


In [11]:
# url을 얻어오지 못한 식당의 리뷰 url 추가
df['naverURL'][1]='https://pcmap.place.naver.com/restaurant/1478444688/review/visitor#'
df

,name,naverURL
0,만년곱창,https://pcmap.place.naver.com/restaurant/14766...
1,홍리마라탕 권선점,https://pcmap.place.naver.com/restaurant/14784...
2,히글리,https://pcmap.place.naver.com/restaurant/18871...
3,레모노이,https://pcmap.place.naver.com/restaurant/19723...


In [14]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.maximize_window()


count = 0 #
current = 0 #현재 진행 상황

goal = len(df['name']) #총 식당 수

#데이터 프레임으로 만들 빈 리스트 생성
rev_list=[]

#문자열 검색을 위한 정규표현식 메서드 호출
pre=re.compile('r"[^\uAC00-\uD7A30-9a-zA-Z\s]"')

for i in range(len(df)): 
    
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    
    
    # 식당 리뷰 개별 url 접속
    driver.get(df['naverURL'][i]) 
    thisurl = df['naverURL'][i]
    time.sleep(2)
    
    #리뷰 더보기 버튼 다 누르기
    while True: 
        try: 
            time.sleep(1) 
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END) 
            time.sleep(3) 
            driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div.lfH3O > a').click() 
            
            time.sleep(2) 
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END) 
            time.sleep(1) 
            
        except NoSuchElementException: 
            print('-모든 리뷰 더보기 완료-') 
            break 
    

     
        
    #리뷰 데이터 스크래핑을 위한 html 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        #키워드 리뷰가 아닌 리뷰글 리스트 검색
        review_lists = soup.select('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div > ul > li')
        print('총 리뷰 수 : ', len(review_lists))
        
        #리뷰 수가 0이 아닌 경우 리뷰 수집
        if len(review_lists) != 0 :
            
            for j, review in enumerate(review_lists):
                
                try:
                    #리뷰 내용이 있는 경우 리뷰 수집
                    driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div > ul > li:nth-child(10) > div.ZZ4OK.IwhtZ > a > span')
                    
                    #내용 더보기가 있는 경우 내용 더보기를 눌러주기
                    try:
                        driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div > ul > li:nth-child(2) > div.ZZ4OK.IwhtZ > a > span.rvCSr > svg').click()
                        time.sleep(1)
                            
                        #리뷰 정보
                        user_review = review.select(' div.ZZ4OK.IwhtZ > a > span')
                        print(user_review)
                        
                                
                        #식당 평균 평점
                        #rating = driver.find_element(by='xpath', value='//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[3]/span[1]/em')
                                
                
                        #리뷰 정보가 있는 경우 식당 이름, 평점, 리뷰 텍스트, 작성 시간을 가져와서 데이터 프레임으로 만들기
                        rev_list.append(
                        [
                            df['name'][i],
                            #rating.text,
                            user_review[0].text,
                            #date[0].text
                        ]
                        )
                                
                        time.sleep(2)            
                
                    
                    
                    except NoSuchElementException:
                        #리뷰 정보
                        user_review = review.select(' div.ZZ4OK.IwhtZ > a > span')
                        print(user_review)
                                
                        
                        #식당 평균 평점
                        #rating = driver.find_element(by='xpath', value='//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[3]/span[1]/em')
                        
        
                        #리뷰 정보가 있는 경우 식당 이름, 평점, 리뷰 텍스트, 작성 시간을 가져와서 데이터 프레임으로 만들기
                        rev_list.append(
                        [
                            df['name'][i],
                            #rating.text,
                            user_review[0].text,
                            #date[0].text
                        ]
                        )            
            
                        time.sleep(2)
                        
                except NoSuchElementException:
                    continue                  
        
       
        


    # 리뷰가 없는 경우        
    except NoSuchElementException: 
               
        rev_list.append(
        [
            df['name'][i],
            #rate,
        ]
        ) 
        time.sleep(2)           
        print("리뷰가 없습니다")

    
            
    #검색한 창 닫고 검색 페이지로 돌아가기    
    # driver.close()
    # driver.switch_to.window(tabs[0])
    print("기본 페이지로 돌아가기")

        
driver.close()

#스크래핑한 데이터를 데이터 프레임으로 만들기
#평점이 없는 경우를 위해 평점 정보 삭제    
column = ["name", "review"]
df2 = pd.DataFrame(rev_list, columns=column)
df2   

진행상황 : 1/4
-모든 리뷰 더보기 완료-
총 리뷰 수 :  2
기본 페이지로 돌아가기
진행상황 : 2/4
-모든 리뷰 더보기 완료-
총 리뷰 수 :  3
기본 페이지로 돌아가기
진행상황 : 3/4
-모든 리뷰 더보기 완료-
총 리뷰 수 :  22
[<span class="zPfVt">매장이 귀엽구 논 커피 메뉴들 다양해서 좋아요. 친절하게 추천해주신거 블루베리들어간 티종류 마신거 같은데 맛있었어요! </span>]
[<span class="zPfVt">회사 친구랑 와서 먹었는데 너무 맛있어요!
재료도 신선하고 든든합니당&gt;&lt;
감바스 포케느낌이에요 ! 새우도 많고 푸짐한데 가성비 짱... </span>]
[<span class="zPfVt">3번째 방문인데 메뉴가 정말 다 맛잇어요!!! 오늘먹은 당근라페ㅠㅠ계속 생각날꺼같아요ㅜㅜ </span>]
[<span class="zPfVt">너무 맛있어욤 ..😶‍🌫️😶‍🌫️ 야채를 싫어하는데 야채들로 이렇게 맛있는 음식이 나올 수 있다니.. ㅠㅠ 깜짝 놀랬어요 다이어트 할때 자주 올게요! (수내동까지 배달 원합니다 ,,🤍)</span>]
[<span class="zPfVt">모든 음식이 맛있어서 다음에 또 오고싶어요!</span>]
[<span class="zPfVt">분위기도 깔끔하니 좋고 음식 너무 맛있어요 커피도 맛있음ㅋㅋㅋ 한번와보고 연달아 세번째와요 오래오래 있어주세영</span>]
[<span class="zPfVt">샌드위치 너무 맛있어요</span>]
[<span class="zPfVt">아담하지만 인테리어도 이쁘고 대화하기 좋아요!</span>]
[<span class="zPfVt">두부샌드 추천</span>]
[<span class="zPfVt">ㅇㅇㅇ</span>]
[]


IndexError: list index out of range